# 设置数据文件存放根目录（不直接放在该项目中，该项目有github仓库，避免大内存数据影响代码推送）

In [ ]:
boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"

# 建立映射表

In [ ]:
import pandas as pd
import os

# ========================
# 路径配置
# ========================
# boot_file_adr = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST"
raw_csv_dir = os.path.join(boot_file_adr, "raw_csv")
save_dir = os.path.join(boot_file_adr, "mapping_table")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "KAIST_vibration_mapping.csv")

# ========================
# 获取文件列表
# ========================
csv_files = sorted([f for f in os.listdir(raw_csv_dir) if f.endswith(".csv")])

# ========================
# 标签映射（Normal=1，不同转速/故障独立编号）
# ========================
mapping_rules = {
    "Normal": 1,
    "BPFI_03": 2,
    "BPFI_10": 3,
    "BPFI_30": 4,
    "BPFO_03": 5,
    "BPFO_10": 6,
    "BPFO_30": 7,
    "Misalign_01": 8,
    "Misalign_03": 9,
    "Misalign_05": 10,
    "Unbalance_0583mg": 11,
    "Unbalance_1169mg": 12,
    "Unbalance_1751mg": 13,
    "Unbalance_2239mg": 14,
    "Unbalance_3318mg": 15,
}

# ========================
# 生成映射表
# ========================
records = []
for f in csv_files:
    matched_label = None
    for key, label in mapping_rules.items():
        if key in f:
            matched_label = label
            break
    if matched_label is None:
        matched_label = -1
    records.append({
        "filename": f,
        "label": matched_label
    })

df = pd.DataFrame(records)
df.to_csv(save_path, index=False)

print(f"✅ 已生成映射表：{save_path}")
display(df)


# 数据混合

## 全类别混合，用于基本模型训练评估

### mixed data->csv

In [ ]:
import os
import pandas as pd

# ===================== 1️⃣ 参数设置 =====================
raw_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/raw_csv"
mapping_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mapping_table/KAIST_vibration_mapping.csv"
save_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass.csv"
used_index_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/used_indices.csv"

# ===================== 2️⃣ 读取映射表 =====================
mapping = pd.read_csv(mapping_path)
print(f"✅ 映射表读取成功，共 {len(mapping)} 条")

# ===================== 3️⃣ 每类真实抽取数量 =====================
sample_dict = { 1: 2000, # normal 
                2: 1000, 
                3: 800,
                4: 900, 
                
                1: 1000,
                5: 500, 
                6: 1500, 
                7: 800,

                1: 2000, 
                8: 900, 
                9: 1500,

                1: 1600, 
                10: 800, 
                11: 900, 
                12: 600, 
                
                1: 800, 
                13: 600, 
                14: 900, 
                }

# ===================== 4️⃣ 已用索引加载 =====================
if os.path.exists(used_index_path):
    used_indices = pd.read_csv(used_index_path)
else:
    used_indices = pd.DataFrame(columns=["source_file", "index"])

# ===================== 5️⃣ 创建输出文件（第一次写入时加表头） =====================
first_write = True
if os.path.exists(save_path):
    os.remove(save_path)  # 避免旧文件干扰
    print("⚙️ 已清空旧的混合文件。")

# ===================== 6️⃣ 分批抽样逻辑 =====================
for fname in os.listdir(raw_dir):
    if not fname.endswith(".csv"): 
        continue
    if "2Nm" in fname:  # 🚫 排除2Nm
        continue

    file_path = os.path.join(raw_dir, fname)
    df = pd.read_csv(file_path)
    
    # 从映射表中获取类别
    label = mapping.loc[mapping["filename"] == fname, "label"].values[0]
    df["label"] = label
    df["source_file"] = fname

    # 查出这个 label 需要抽取多少样本
    if label not in sample_dict:
        print(f"⚠️ {fname} (label={label}) 未在 sample_dict 中，跳过。")
        continue
    need_n = sample_dict[label]
    
    # 从当前文件中过滤掉已使用过的数据
    used_idx = used_indices.loc[used_indices["source_file"] == fname, "index"].tolist()
    df_avail = df.loc[~df.index.isin(used_idx)]

    # 如果当前类别的数据已经够了，跳过
    already_have = used_indices["source_file"].isin(mapping.loc[mapping["label"] == label, "filename"]).sum()
    if already_have >= need_n:
        print(f"✅ 类别 {label} 已抽取足够样本 ({already_have} >= {need_n})，跳过 {fname}")
        continue

    # 当前还需的数量
    remain = need_n - already_have
    if remain <= 0:
        continue

    # 当前文件可用数量
    take_n = min(remain, len(df_avail))
    if take_n == 0:
        continue

    sampled_df = df_avail.sample(n=take_n, random_state=42)
    
    # 写入结果文件（追加模式）
    sampled_df.to_csv(save_path, mode="a", header=first_write, index=False)
    first_write = False

    # 更新 used_indices
    new_used = pd.DataFrame({
        "source_file": fname,
        "index": sampled_df.index
    })
    used_indices = pd.concat([used_indices, new_used], ignore_index=True)

    print(f"📦 已从 {fname} (label={label}) 抽取 {take_n} 条，剩余 {remain - take_n} 条待补。")

# ===================== 7️⃣ 保存使用记录 =====================
used_indices.to_csv(used_index_path, index=False)
print(f"✅ 混合数据逐步保存完成：{save_path}")
print(f"📁 已更新使用记录：{used_index_path}")


### mixed_data->pyg

#### 时间邻点建图

In [ ]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_local_temporal_graph
import pandas as pd
import os

# ================== 2️⃣ 参数设置 ==================
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass.csv"
save_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time"
num_edges = 10  # 每个节点的边数（上下平均分配）

# ================== 3️⃣ 预处理：忽略最后一列 ==================
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ 找不到混合文件: {csv_path}")

df = pd.read_csv(csv_path)
print(f"📊 原始数据形状: {df.shape}")

# 去掉最后一列（通常为标签）
df = df.iloc[:, :-1]
print(f"🧹 已移除最后一列后形状: {df.shape}")

# 保存临时文件供建图使用
temp_csv_path = csv_path.replace(".csv", "_nofinal.csv")
df.to_csv(temp_csv_path, index=False)
print(f"✅ 已生成去除标签列的临时文件: {temp_csv_path}")

# ================== 4️⃣ 调用函数建图 ==================
nodes_csv, edges_csv, graph_pt = build_local_temporal_graph(
    csv_path=temp_csv_path,
    save_dir=save_dir,
    num_edges=num_edges
)

# ================== 5️⃣ 输出结果路径 ==================
print("\n✅ 局部时序图已构建完成：")
print(f"📁 节点文件: {nodes_csv}")
print(f"📁 边文件:   {edges_csv}")
print(f"📁 图文件:   {graph_pt}")



✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
📊 原始数据形状: (12500, 7)
🧹 已移除最后一列后形状: (12500, 6)
✅ 已生成去除标签列的临时文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv
✅ 图结构构建完成，共 12500 个节点，125000 条无向边
📁 nodes.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/nodes.csv
📁 edges.csv: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/edges.csv
📁 graph.pt : /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/graph.pt
🧩 特征维度: 4 (已自动忽略首列与标签列)

✅ 局部时序图已构建完成：
📁 节点文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/nodes.csv
📁 边文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/edges.csv
📁 图文件:   /home/charles/HZU/Data_processed/my

In [5]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import add_random_masks_with_label_split
import pandas as pd
import os
import torch

# ================== 2️⃣ 参数设置 ==================
ratios = {"train": 0.6, "val": 0.2, "test": 0.2}

graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/graph.pt"
save_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/mixed_allclass_arabic_graph_withmask.pt"

# ================== 3️⃣ 调用函数生成掩码 ==================
print("🚀 开始为图添加掩码 ...")
new_graph_path = add_random_masks_with_label_split(
    graph_path=graph_path,
    save_path=save_path,      # 若想覆盖源文件则直接传 None
    ratios=ratios,
    train_label_ratio=0.4,    # 训练集中40%为有标签
)
print("\n✅ 掩码生成与保存完成。")

# ================== 4️⃣ 加载验证 ==================
print("\n🧩 验证生成的图结构与掩码 ...")
data = torch.load(new_graph_path)

# === 基本结构 ===
print(f"\n📦 图数据结构: {data}")
print(f"x shape: {data.x.shape}, y shape: {data.y.shape}")
print(f"前5个标签: {data.y[:5].tolist()}")

# === 掩码存在性检查 ===
mask_attrs = ["train_mask", "val_mask", "test_mask", "train_withlabel_mask", "train_nolabel_mask"]
for attr in mask_attrs:
    exists = hasattr(data, attr)
    print(f"{attr:<25}: {'✅ 存在' if exists else '❌ 不存在'}")

# === 掩码分布验证 ===
if hasattr(data, "train_mask"):
    total_nodes = data.x.size(0)
    n_train = int(data.train_mask.sum())
    n_val = int(data.val_mask.sum()) if hasattr(data, "val_mask") else 0
    n_test = int(data.test_mask.sum()) if hasattr(data, "test_mask") else 0
    n_withlabel = int(data.train_withlabel_mask.sum()) if hasattr(data, "train_withlabel_mask") else 0
    n_nolabel = int(data.train_nolabel_mask.sum()) if hasattr(data, "train_nolabel_mask") else 0

    print("\n📊 掩码数量验证：")
    print(f"  train_mask         = {n_train}")
    print(f"  val_mask           = {n_val}")
    print(f"  test_mask          = {n_test}")
    print(f"  └─ train_withlabel = {n_withlabel}")
    print(f"  └─ train_nolabel   = {n_nolabel}")
    print(f"  合计节点数         = {total_nodes}")
    if n_train + n_val + n_test == total_nodes:
        print("✅ 掩码划分总数匹配节点总数")
    else:
        print("⚠️ 注意：掩码划分节点数与总数不完全匹配，可能存在舍入误差。")

print("\n🎯 掩码验证完成。")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
🚀 开始为图添加掩码 ...
📦 已加载图文件，共 12500 个节点。
✅ 已生成主掩码：train=7500, val=2500, test=2500
🎯 训练集内部划分：with_label=3000, no_label=4500
💾 已将含掩码的新图文件保存至：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/time/mixed_allclass_arabic_graph_withmask.pt

✅ 掩码生成与保存完成。

🧩 验证生成的图结构与掩码 ...

📦 图数据结构: Data(x=[12500, 4], edge_index=[2, 250000], y=[12500], train_mask=[12500], val_mask=[12500], test_mask=[12500], train_withlabel_mask=[12500], train_nolabel_mask=[12500])
x shape: torch.Size([12500, 4]), y shape: torch.Size([12500])
前5个标签: [1, 1, 1, 1, 1]
train_mask               : ✅ 存在
val_mask                 : ✅ 存在
test_mask                : ✅ 存在
train_withlabel_mask     : ✅ 存在
train_nolabel_mask       : ✅ 存在

📊 掩码数量验证：
  train_mask         = 7500
  val_mask       

/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib/build_pyg_data.py:278: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

#### 余弦相似度建图

In [6]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import build_similarity_knn_graph
import pandas as pd
import torch
import os

# ================== 2️⃣ 参数设置 ==================
csv_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/mixed_allclass_nofinal.csv"
save_dir = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn"

# ================== 3️⃣ 调用建图函数 ==================
print("🚀 开始构建基于余弦相似度的KNN图 ...")
nodes_csv, edges_csv, graph_pt = build_similarity_knn_graph(
    csv_path=csv_path,
    save_dir=save_dir,
    num_edges=10
)
print("\n✅ 图结构已生成：")
print(f"📁 节点文件: {nodes_csv}")
print(f"📁 边文件:   {edges_csv}")
print(f"📁 图文件:   {graph_pt}")

# ================== 4️⃣ 验证生成的图数据 ==================
print("\n🧩 验证生成的 PyG 图数据结构 ...")

# 读取 PyG 对象
data = torch.load(graph_pt, weights_only=False)

# === 基础信息 ===
num_nodes = data.x.size(0)
num_edges = data.edge_index.size(1)
num_features = data.x.size(1)
has_label = hasattr(data, "y") and data.y is not None

print(f"📦 Data 对象结构: {data}")
print(f"🧮 节点数: {num_nodes}")
print(f"🧮 边数: {num_edges // 2}（无向边）")
print(f"🧩 特征维度: {num_features}")
print(f"🏷️  标签存在: {'✅ 是' if has_label else '❌ 否'}")

# === 标签验证 ===
if has_label:
    y = data.y.cpu().numpy()
    unique_labels = sorted(set(y))
    print(f"🎯 标签类别数量: {len(unique_labels)} | 类别: {unique_labels[:10]}")
    print(f"前5个标签: {y[:5]}")
else:
    print("⚠️ 未检测到 data.y，标签列可能缺失或未正确指定。")

# === 节点/边文件验证 ===
print("\n📊 验证节点和边 CSV 文件 ...")
df_nodes = pd.read_csv(nodes_csv)
df_edges = pd.read_csv(edges_csv)

print(f"✅ 节点文件行数: {len(df_nodes)} | 与图节点数 {'一致 ✅' if len(df_nodes)==num_nodes else '不一致 ⚠️'}")
print(f"✅ 边文件行数: {len(df_edges)} | 每两行对应一条无向边")

if "source" in df_edges.columns and "target" in df_edges.columns:
    unique_nodes = pd.concat([df_edges["source"], df_edges["target"]]).nunique()
    print(f"🧮 边中涉及的唯一节点数: {unique_nodes}")
else:
    print("⚠️ 边文件缺少 'source' 或 'target' 列！")

print("\n🎯 图数据验证完成。")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
🚀 开始构建基于余弦相似度的KNN图 ...
📊 已读取数据: (12500, 6)
🧩 使用特征列数: 4 | 特征列示例: ['vibration_ch1', 'vibration_ch2', 'vibration_ch3', 'vibration_ch4'] ...
⚙️ 正在计算余弦相似度矩阵...


🔍 正在为每个节点选取 10 个最相似邻居...
✅ 图构建完成，共 12500 个节点，125000 条无向边。
📁 节点文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_nodes.csv
📁 边文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_edges.csv
📁 图文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_graph.pt

✅ 图结构已生成：
📁 节点文件: /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_nodes.csv
📁 边文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_edges.csv
📁 图文件:   /home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_graph.pt

🧩 验证生成的 PyG 图数据结构 ...
📦 Data 对象结构: Data(x=[12500, 4], edge_index=[2, 250000], y=[12500])
🧮 节点数: 12500
🧮 边数: 125000（无向边）
🧩 特征维度: 4
🏷️  标签存在: ✅ 是
🎯 标签类别数量: 14 | 类别: [np.int64(1), np.int64(2), np.int

In [7]:
# ================== 1️⃣ 导入路径与函数 ==================
%run ../_init_path.py
from build_pyg_data import add_random_masks_with_label_split
import pandas as pd
import torch
import os

# ================== 2️⃣ 参数设置 ==================
ratios = {"train": 0.6, "val": 0.2, "test": 0.2}

graph_path = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_nofinal_graph.pt"
save_path  = "/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt"

# ================== 3️⃣ 生成掩码 ==================
print("🚀 开始为图添加随机掩码 ...")
new_graph_path = add_random_masks_with_label_split(
    graph_path=graph_path,
    save_path=save_path,     # 若想覆盖源文件则直接传 None
    ratios=ratios,
    train_label_ratio=0.4,   # 训练集中40%为有标签
)
print(f"\n✅ 掩码生成与保存完成：{new_graph_path}")

# ================== 4️⃣ 验证生成结果 ==================
print("\n🧩 验证生成的图结构与掩码 ...")

# 读取 PyG 数据
data = torch.load(new_graph_path, weights_only=False)

# === 基本信息 ===
print(f"\n📦 图数据结构: {data}")
print(f"x shape: {data.x.shape}, y shape: {data.y.shape}")
print(f"前5个标签: {data.y[:5].tolist()}")

# === 掩码存在性检测 ===
mask_attrs = ["train_mask", "val_mask", "test_mask", "train_withlabel_mask", "train_nolabel_mask"]
print("\n📋 掩码存在性检查：")
for attr in mask_attrs:
    print(f"  {attr:<25}: {'✅ 存在' if hasattr(data, attr) else '❌ 不存在'}")

# === 掩码统计 ===
if hasattr(data, "train_mask"):
    total_nodes = data.x.size(0)
    n_train = int(data.train_mask.sum())
    n_val   = int(data.val_mask.sum()) if hasattr(data, "val_mask") else 0
    n_test  = int(data.test_mask.sum()) if hasattr(data, "test_mask") else 0
    n_with  = int(data.train_withlabel_mask.sum()) if hasattr(data, "train_withlabel_mask") else 0
    n_nol   = int(data.train_nolabel_mask.sum()) if hasattr(data, "train_nolabel_mask") else 0

    print("\n📊 掩码数量验证：")
    print(f"  train_mask         = {n_train}")
    print(f"  val_mask           = {n_val}")
    print(f"  test_mask          = {n_test}")
    print(f"  └─ train_withlabel = {n_with}")
    print(f"  └─ train_nolabel   = {n_nol}")
    print(f"  合计节点数         = {total_nodes}")
    if n_train + n_val + n_test == total_nodes:
        print("✅ 掩码划分总数匹配节点总数")
    else:
        print("⚠️ 注意：掩码划分节点数与总数不完全匹配，可能存在舍入误差。")

# === 标签统计 ===
unique_labels = sorted(set(data.y.cpu().numpy().tolist()))
print("\n🎯 标签分布检查：")
print(f"  标签类别数: {len(unique_labels)}")
print(f"  类别示例: {unique_labels[:10]}")

print("\n🎯 掩码与标签验证完成。")


✅ 已添加路径：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST
✅ 已添加 my_lib：/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib
🚀 开始为图添加随机掩码 ...
📦 已加载图文件，共 12500 个节点。
✅ 已生成主掩码：train=7500, val=2500, test=2500
🎯 训练集内部划分：with_label=3000, no_label=4500
💾 已将含掩码的新图文件保存至：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt

✅ 掩码生成与保存完成：/home/charles/HZU/Data_processed/my_CIL_V1/KAIST/mixed_allclass/pyg_data/cos_knn/mixed_allclass_arabic_graph_withmask.pt

🧩 验证生成的图结构与掩码 ...

📦 图数据结构: Data(x=[12500, 4], edge_index=[2, 250000], y=[12500], train_mask=[12500], val_mask=[12500], test_mask=[12500], train_withlabel_mask=[12500], train_nolabel_mask=[12500])
x shape: torch.Size([12500, 4]), y shape: torch.Size([12500])
前5个标签: [1, 1, 1, 1, 1]

📋 掩码存在性检查：
  train_mask               : ✅ 存在
  val_mask                 : ✅ 存在
  test_mask

/home/charles/HZU/Industrial_Software_Testing/Industrial_Software_Testing/my_CIL_V1/data_process/KAIST/my_lib/build_pyg_data.py:278: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

#### time + cos_knn 融合建图